# 0. Importar liberías y rutas de archivo

In [2]:
import pandas as pd
import numpy as np
import matplotlib
from pathlib import Path

In [3]:
ruta_script = Path("DS Market prediction.ipynb").resolve().parent
ruta_archivo_ventas = ruta_script / "Datos" / "item_sales.csv"
ruta_archivo_precios = ruta_script /"Datos"/"item_prices.csv"
ruta_archivo_fechas = ruta_script /"Datos"/"daily_calendar_with_events.csv"

In [4]:
sales = pd.read_csv(ruta_archivo_ventas)
calendar = pd.read_csv(ruta_archivo_fechas)
prices = pd.read_csv(ruta_archivo_precios)

# 1. Análisis inicial

Hay que realizar un powerbi para el analisis de los datos. Para ello, en primer lugar, debemos saber con que estamos trabajando y modelar las tablas para sacar las correctas visualizaciones en powerbi.
Con una rápida exploración de datos, necesitamos:
- Saber el número de tiendas.
- Saber el número de categorías.
- Saber el número de productos.
- Trasponer las ventas para que cada día corresponda a un registro.
- Asignar cada día a su semana correspondiente para saber el precio del producto en cada registro.

# 2. Preparación de limpieza de datos para su visualización

## 2.1. Tabla calendar

In [5]:
calendar.head()

,date,weekday,weekday_int,d,event
0,2011-01-29,Saturday,1,d_1,NaN
1,2011-01-30,Sunday,2,d_2,NaN
2,2011-01-31,Monday,3,d_3,NaN
3,2011-02-01,Tuesday,4,d_4,NaN
4,2011-02-02,Wednesday,5,d_5,NaN


In [6]:
calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1913 entries, 0 to 1912
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         1913 non-null   object
 1   weekday      1913 non-null   object
 2   weekday_int  1913 non-null   int64 
 3   d            1913 non-null   object
 4   event        26 non-null     object
dtypes: int64(1), object(4)
memory usage: 74.9+ KB


In [11]:
print("Calendar",calendar.isnull().sum())

Calendar date              0
weekday           0
weekday_int       0
d                 0
event          1887
dtype: int64


Observamos que solo hay nulos en eventos, porque no todas las fechas tienen un evento.

Primero, hay que formatear las fechas 

In [16]:
calendar['date'] = pd.to_datetime(calendar['date'])

# De paso probamos extrer año, mes y dia si es necesario
calendar['year'] = calendar['date'].dt.year
calendar['months'] = calendar['date'].dt.month
calendar['day'] = calendar['date'].dt.day

# Convertimos d a un int que se pueda cruzar
calendar["d"] = calendar["d"].str.extract(r'(\d+)').astype(int)

In [17]:
calendar.head()

,date,weekday,weekday_int,d,event,year,months,day
0,2011-01-29,Saturday,1,1,NaN,2011,1,29
1,2011-01-30,Sunday,2,2,NaN,2011,1,30
2,2011-01-31,Monday,3,3,NaN,2011,1,31
3,2011-02-01,Tuesday,4,4,NaN,2011,2,1
4,2011-02-02,Wednesday,5,5,NaN,2011,2,2


## 2.2. Tabla prices

In [7]:
# Valores Nulos por cada Dataset
print('Valores nulos por cada Dataset:')
print("Calendar:", calendar.isnull().sum(), '\n')
print("prices:", prices.isnull().sum(), '\n')
print("sales:", sales.isnull().sum(), '\n')

Valores nulos por cada Dataset:
Calendar: date              0
weekday           0
weekday_int       0
d                 0
event          1887
dtype: int64 

prices: item               0
category           0
store_code         0
yearweek      243920
sell_price         0
dtype: int64 

sales: id            0
item          0
category      0
department    0
store         0
             ..
d_1909        0
d_1910        0
d_1911        0
d_1912        0
d_1913        0
Length: 1920, dtype: int64 



In [ ]:
# Valores Unicos en columnas claves
print("Categorias de productos en ventas:", sales['category'].unique())
print("Categorias de productos en precios:", prices['category'].unique())

In [ ]:
sales.head()

📌 Formatear fechas


📌 Corregir valores en yearweek


In [ ]:
prices.head()
# yearweek me la saco como una fecha :4, para recorrerlo tiene que estar como str

In [ ]:
prices.info()

In [118]:
# yearweek a string, pero asegurarnos de que no haya NaN
prices['yearweek'] = prices['yearweek'].astype(str)

In [85]:
# Reemplazar valores "nan" explícitos y valores NaN reales por "000000"
prices['yearweek'] = prices['yearweek'].replace('nan', '000000').fillna('000000')

In [86]:
# Extraer año y semana correctamente
prices['year'] = prices['yearweek'].str[:4].astype(int, errors='ignore')
prices['week'] = prices['yearweek'].str[4:].astype(int, errors='ignore')

In [ ]:
print(prices[['yearweek', 'year', 'week']].head())

In [ ]:
# Asegurar que no haya NaN en 'yearweek' y convertir a int sin errores
prices['yearweek'] = prices['yearweek'].fillna(0).astype(float).astype(int).astype(str)

# Verificar resultado
print(prices[['yearweek', 'year', 'week']].head())

In [ ]:
print("Calendario")
print(calendar.info(), "\n")

print("Precios")
print(prices.info(), "\n")

print("Ventas")
print(sales.info(), "\n")


Normalizar Nombres de columnas

In [90]:
calendar.columns = calendar.columns.str.lower().str.replace(' ', '_')
prices.columns = prices.columns.str.lower().str.replace(' ', '_')
sales.columns = sales.columns.str.lower().str.replace(' ', '_')


In [ ]:
print("Valores nulos en precios:\n", prices.isnull().sum())
print("Valores nulos en ventas:\n", sales.isnull().sum())
print("Valores nulos en calendario:\n", calendar.isnull().sum())

In [92]:
prices.fillna(0, inplace=True)
sales.fillna(0, inplace=True)
calendar.fillna("Sin evento", inplace=True)  # Para la columna 'event'

Convertir fechas a formato estándar

In [93]:
calendar['date'] = pd.to_datetime(calendar['date'])

Reesturar Sales

In [ ]:
# Extraer solo los números de la columna 'd' y convertir a int
calendar['d'] = calendar['d'].str.extract(r'(\d+)').astype(float).fillna(0).astype(int)

# Verificar resultado
print(calendar[['d']].head())

In [95]:
# Volver a convertir ventas de formato wide a formato long
sales_melted = sales.melt(
    id_vars=['id', 'item', 'category', 'department', 'store', 'store_code', 'region'], 
    var_name='day', 
    value_name='units_sold',
)

In [ ]:
sales_melted

In [ ]:
len(sales)

In [ ]:
chunk_size = 30490  # Procesa 5 millones de filas por lote

sales_chunks = np.array_split(sales, len(sales) // chunk_size)

sales_melted_chunks = []
for chunk in sales_chunks:
    melted_chunk = chunk.melt(
        id_vars=['id', 'item', 'category', 'department', 'store', 'store_code', 'region'], 
        var_name='day', 
        value_name='units_sold'
    )
    sales_melted_chunks.append(melted_chunk)

sales_melted = pd.concat(sales_melted_chunks, ignore_index=True)